In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os


from scipy.fft import fft, fftshift, fftfreq
from scipy.signal.windows import hamming

from scipy.spatial import cKDTree
from scipy.spatial.distance import pdist, squareform
from scipy import stats
from sklearn.neighbors import NearestNeighbors
from math import log



In [ ]:
plt.rcParams.update({
    'figure.figsize': (12, 8),  # Change figure size
    'lines.linewidth': 2,      # Change line width
    'axes.labelsize': 38,      # Change label font size
    'axes.titlesize': 40,      # Change title font size
    'legend.fontsize': 30,     # Change legend font size
    'xtick.labelsize': 36,     # Change x-axis tick font size
    'ytick.labelsize': 36,      # Change y-axis tick font size
    'figure.autolayout': True  # Automatically adjust layout to fit labels
})


In [ ]:


def cross_correlation(P1, P2, max_lag):
    if len(P1) != len(P2):
        raise ValueError("P1 and P2 must have the same length")
    
    n = len(P1)
    mean_P1 = np.mean(P1)
    mean_P2 = np.mean(P2)

    lags = np.arange(-max_lag, max_lag + 1)
    C = np.zeros(len(lags))

    for i, Δt in enumerate(lags):
        if Δt >= 0:
            num = np.mean((P1[:n-Δt] - mean_P1) * (P2[Δt:] - mean_P2))
        else:
            num = np.mean((P1[-Δt:] - mean_P1) * (P2[:n+Δt] - mean_P2))
        denom = np.sqrt(np.mean((P1 - mean_P1)**2) * np.mean((P2 - mean_P2)**2))
        C[i] = num / denom

    return lags, C


In [ ]:
def sliding_window_correlation_previous(dataset1, dataset2, delta_t, window_size, step_size):
    """
    Compute sliding window correlation between two signals with a delay applied to dataset1.
    
    Parameters:
    - dataset1: np.array, the slave (lagging) dataset
    - dataset2: np.array, the master (leading) dataset
    - delta_t: int, the delay (in samples)
    - window_size: int, number of samples in each window
    - step_size: int, step size for the sliding window

    Returns:
    - indices: np.array of int, window start indices
    - correlations: np.array of float, absolute correlation values per window
    """
    dataset1 = np.asarray(dataset1)
    dataset2 = np.asarray(dataset2)

    n = len(dataset1)
    start_indices = np.arange(0, n - window_size - delta_t + 1, step_size)
    correlations = np.empty(len(start_indices))

    for idx, i in enumerate(start_indices):
        window1 = dataset1[i : i + window_size]
        window2 = dataset2[i + delta_t : i + delta_t + window_size]
        correlations[idx] = abs(np.corrcoef(window1, window2)[0, 1])

    return start_indices, correlations

In [ ]:
%matplotlib widget

# Temporal trace

In [ ]:
Δf_list = np.arange(-30,31)
Δf_list = np.round(Δf_list, 1)  # Round to 2 decimal place    

for Δf in Δf_list:
    # Load the CSV file into a DataFrame
    file_path = os.path.join(rf"C:\Users\magic\OneDrive\Desktop\Programas\Julia_programs\beca_jae_julia\Lang-kobayashi\four_lasers_simulations\results\two_lasers\results_laser_1_vs_laser_1_same_delay\laser_simulation_Δf_{Δf}_GHz_laser_1_laser_1_lab_same_delay_new_k_0.15.csv")
    df = pd.read_csv(file_path)
    
    # Clean and convert Julia complex strings to Python complex
    def parse_julia_complex(s):
        try:
            s = s.replace("im", "j").replace(" ", "")
            return complex(s)
        except:
            return np.nan  # or raise if you want to catch malformed ones


    # Apply to each column
    t = df["time"].values
    E1 = np.array([parse_julia_complex(x) for x in df["E1"]])
    E2 = np.array([parse_julia_complex(x) for x in df["E2"]])

    # Compute intensities
    S1 = np.abs(E1)**2
    S2 = np.abs(E2)**2

    index_steady = np.argmax(t > 1900e-9) -1
    t_steady = t[index_steady:] * 10**(9)
    S1_steady = S1[index_steady:]
    S2_steady = S2[index_steady:] 

    index_steady_2 = np.argmax(t > 1990e-9) -1
    t_steady_2 = t[index_steady_2:] * 10**(9)
    S1_steady_2 = S1[index_steady_2:]
    S2_steady_2 = S2[index_steady_2:]

    index_transient = np.argmax(t >= 30e-9) - 1
    t_transient = t[:index_transient] * 10**(9)
    S1_transient = S1[:index_transient] 
    S2_transient = S2[:index_transient] 

    t = t * 10**(9)
    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(t, S1, label="$S_1$")
    plt.plot(t, S2, label="$S_2$")
    plt.xlabel("t (ns)")
    plt.ylabel("S")
    plt.title(rf"{Δf}")
    plt.legend()
    #plt.savefig("trace_laser_1_vs_laser_1_diff_delay_noise_k_0.15_deltaf_0_GHz.png")

In [ ]:
Δf_list = np.arange(-10,0,0.5)
Δf_list = np.round(Δf_list, 1)  # Round to 2 decimal place    

for Δf in Δf_list:
    # Load the CSV file into a DataFrame
    file_path = os.path.join(rf"/home/adam/vscode/python/Infolante/traces/results_4_laser_new_params_larger_wait_increased_alpha_1/4_lasers_simulation_Δf_{Δf}_RK4_no_noise_new_params_larger_wait.csv")
    df = pd.read_csv(file_path)

    # Clean and convert Julia complex strings to Python complex
    def parse_julia_complex(s):
        try:
            s = s.replace("im", "j").replace(" ", "")
            return complex(s)
        except:
            return np.nan  # or raise if you want to catch malformed ones


    # Apply to each column
    t = df["time"].values
    E1 = np.array([parse_julia_complex(x) for x in df["E1"]])
    E2 = np.array([parse_julia_complex(x) for x in df["E2"]])
    E3 = np.array([parse_julia_complex(x) for x in df["E3"]])
    E4 = np.array([parse_julia_complex(x) for x in df["E4"]])

    # Compute intensities
    S1 = np.abs(E1)**2
    S2 = np.abs(E2)**2
    S3 = np.abs(E3)**2
    S4 = np.abs(E4)**2


    t = t * 10**(9)
    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(t, S1, label="$S_1$")
    plt.plot(t, S2, label="$S_2$")
    plt.plot(t, S3, label="$S_3$")
    plt.plot(t, S4, label="$S_4$")
    plt.xlabel("t (ns)")
    plt.ylabel("S")
    plt.title(rf"{Δf}")
    #plt.legend()
    #plt.savefig(rf"trace_laser_1_vs_laser_1_diff_delay_noise_k_0.15_deltaf_{Δf}_GHz.png")
    #plt.close()

# RHF and optical spectrum

In [ ]:
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
import numpy as np

Here we estimate the relaxation frequency $\nu_{R}$ of the lasers

In [ ]:



def plot_function_delay_laser():
    # === Load CSV ===lasers_simulations\4_la
    
    file_path = os.path.join(rf"D:\four_lasers\4_lasers_simulation_Δf_0_all_k_0.1_freq_missmatch_laser_1_k_0.15_solitary_laser.csv")
    df = pd.read_csv(file_path)

    # === Time vector and steady-state cut ===
    t = df["time"].values
    index_steady = np.argmax(t > 10e-9)
    t = t[index_steady:]
    dt = t[1] - t[0]

    # === Complex conversion helper ===
    def parse_julia_complex(x):
        return complex(x.replace("im", "j").replace(" ", ""))

    # === Parse and slice complex fields ===
    E_fields = {}
    for i in range(1, 5):
        col = f"E{i}"
        E_fields[col] = np.array([parse_julia_complex(val) for val in df[col]])[index_steady:]

    # === Apply Hamming window ===
    window = hamming(len(t))

    # === FFT of intensities (|E|^2) ===
    freqs = fftshift(fftfreq(len(t), dt)) / 1e9  # GHz
    valid = freqs > 0.01

    plt.figure()
    for i in range(1, 5):
        E = np.abs(E_fields[f"E{i}"])**2
        spectrum = np.abs(fftshift(fft(E * window)))
        spectrum = 10*np.log10(spectrum)
        plt.plot(freqs[valid], spectrum[valid], label=f"Laser {i}")
    #plt.yscale('log')
    plt.xlabel(r"$\nu$ (GHz)")
    plt.ylabel(r"Intensity (dB)")
    plt.legend()
    #plt.savefig(os.path.join(os.getcwd(), "RFS_4_lasers_Δf_-7.6_GHz_noise.png"))

    # === Zoomed view of above ===
    plt.figure()
    for i in range(1, 5):
        E = np.abs(E_fields[f"E{i}"])**2
        spectrum = np.abs(fftshift(fft(E * window)))
        spectrum = 10*np.log10(spectrum)
        plt.plot(freqs[valid], spectrum[valid], label=f"Laser {i}")
    #plt.yscale('log')
    plt.xlim([0, 20])
    plt.xlabel(r"$\nu$ (GHz)")
    plt.ylabel(r"Intensity (dB)")
    plt.legend()
    #plt.savefig(os.path.join(os.getcwd(), "RFS_4_lasers_zoom_Δf_-7.6_GHz_noise.png"))

    # === FFT of complex fields (optical spectra) ===
    plt.figure()
    for i in range(1, 5):
        E_complex = E_fields[f"E{i}"]
        spectrum_c = np.abs(fftshift(fft(E_complex * window)))
        db_spectrum = 20 * np.log10(spectrum_c)
        plt.plot(freqs, db_spectrum, label=f"Laser {i}")
    plt.xlim([-20, 20])
    plt.xlabel(r"$\nu$ (GHz)")
    plt.ylabel(r"Intensity (dB)")
    plt.legend()
    #plt.savefig(os.path.join(os.getcwd(), "optical_spectra_4_lasers_Δf_-7.6_GHz_noise.png"))

    
    relaxation_freqs = {}
    relaxation_errors = {}

    for i in range(1, 5):
        E = np.abs(E_fields[f"E{i}"])**2
        spectrum = np.abs(fftshift(fft(E * window)))
        spectrum = 10*np.log(spectrum)

        # Mask
        spectrum_valid = spectrum[valid]
        freqs_valid = freqs[valid]

        # Find peaks
        peaks, _ = find_peaks(spectrum_valid, height=-70)  # Only consider visible peaks

        if peaks.size > 0:
            peak_idx = peaks[np.argmax(spectrum_valid[peaks])]
            peak_freq = freqs_valid[peak_idx]

            # Estimate error using FWHM (crude Gaussian-like assumption)
            half_max = spectrum_valid[peak_idx] - 3  # ~3 dB down from peak
            # Interpolate curve around peak to find where it drops to half max
            interp_func = interp1d(freqs_valid, spectrum_valid, kind='linear')
            freqs_fine = np.linspace(freqs_valid[0], freqs_valid[-1], 10000)
            spectrum_fine = interp_func(freqs_fine)
            above_half = freqs_fine[spectrum_fine >= half_max]

            if len(above_half) >= 2:
                fwhm = above_half[-1] - above_half[0]
                error = fwhm / 2  # crude ± estimate
            else:
                error = None

            relaxation_freqs[i] = peak_freq
            relaxation_errors[i] = error

            print(f"{i}: Relaxation Frequency = {peak_freq}±{error} GHz")



plot_function_delay_laser()

In [ ]:

def plot_function_delay_laser():
    # === Load CSV ===lasers_simulations\4_la
    Δf_list = np.arange(-10,1,0.5)
    Δf_list = np.round(Δf_list, 1)  # Round to 2 decimal place    

    for Δf in Δf_list:
        # Load the CSV file into a DataFrame
        file_path = os.path.join(rf"/home/adam/vscode/python/Infolante/traces/results_4_lasers_new_params_larger_wait/4_lasers_simulation_Δf_{Δf}_RK4_no_noise_new_params_larger_wait.csv")
        df = pd.read_csv(file_path)

        # === Time vector and steady-state cut ===
        t = df["time"].values
        index_steady = np.argmax(t > 19600e-9)
        t = t[index_steady:]
        dt = t[1] - t[0]

        # === Complex conversion helper ===
        def parse_julia_complex(x):
            return complex(x.replace("im", "j").replace(" ", ""))

        # === Parse and slice complex fields ===
        E_fields = {}
        for i in range(1, 3):
            col = f"E{i}"
            E_fields[col] = np.array([parse_julia_complex(val) for val in df[col]])[index_steady:]

        # === Apply Hamming window ===
        window = hamming(len(t))

    # === FFT of intensities (|E|^2) ===
        freqs = fftshift(fftfreq(len(t), dt)) / 1e9  # GHz
        valid = freqs > 0.01

        plt.figure()
        for i in range(1, 3):
            E = np.abs(E_fields[f"E{i}"])**2
            spectrum = np.abs(fftshift(fft(E * window)))
            spectrum = 10 *np.log(spectrum)
            plt.plot(freqs[valid], spectrum[valid], label=f"Laser {i}")
        plt.xlabel(r"$\nu$ (GHz)")
        plt.ylabel(r"$|FFT(|E(t)|^2)|$")
        plt.legend()
        #plt.savefig(os.path.join(os.getcwd(), "RHF_laser_1_laser_1_Δf_0.02_GHz.png"))

        # === Zoomed view of above ===
        plt.figure()
        for i in range(1, 3):
            E = np.abs(E_fields[f"E{i}"])**2
            spectrum = np.abs(fftshift(fft(E * window)))
            spectrum = 10 * np.log(spectrum)
            plt.plot(freqs[valid], spectrum[valid], label=f"Laser {i}")
        plt.xlim([0, 20])
        plt.xlabel(r"$\nu$ (GHz)")
        plt.ylabel(r"$|FFT(|E(t)|^2)|$")
        plt.legend()
        #plt.savefig(os.path.join(os.getcwd(), "RHF_laser_1_laser_1_zoom_Δf_0.02_GHz.png"))

        # === FFT of complex fields (optical spectra) ===
        plt.figure()
        for i in range(1, 3):
            E_complex = E_fields[f"E{i}"]
            spectrum_c = np.abs(fftshift(fft(E_complex * window)))
            db_spectrum = 20 * np.log10(spectrum_c)
            plt.plot(freqs, db_spectrum, label=f"Laser {i}")
        plt.title(rf"{Δf}")
        plt.xlabel(r"$\nu$ (GHz)")
        plt.ylabel(r"$|FFT(E_r)|$ (dB)")
        plt.legend()
        #plt.savefig(os.path.join(os.getcwd(), "optical_spectra_laser_1_laser_1_Δf_0.02_GHz.png"))

plot_function_delay_laser()

# Cross-correlations

## All the $C_{\text{max}}$

In [ ]:

def parse_julia_complex(x):
    return complex(x.replace("im", "j").replace(" ", ""))

def analyze_cross_correlations():
        
    Δf_list = np.arange(-30, 30.1, 0.1)
    Δf_list = np.round(Δf_list, 1)  # Round to 1 decimal place
    for element in Δf_list:
        if element == -0.0:
            index = np.where(Δf_list == element)[0][0]
            Δf_list[index] = 0.0  # Change -0.0 to 0.0

    c_max_list = []
    lag_at_cmax_list = []
    pairs_list = []

    current_dir = os.getcwd()
    h = 0.2e-12
    Δt = 50
    tau = 1e-8
    lag_max_index = int(round(10*tau / (h * Δt)))  # max lag index (samples)

    for Δf in Δf_list:
        print(Δf)
        file_path = os.path.join(rf"/home/adam/vscode/python/Infolante/traces/results_4_lasers_new_params_larger_wait/4_lasers_simulation_Δf_{Δf}_RK4_no_noise_new_params_larger_wait.csv")

        df = pd.read_csv(file_path)
        t = df["time"].values
        index_steady = np.argmax(t > 1800e-9)

        x1 = np.abs(np.array([parse_julia_complex(val) for val in df["E1"].values[index_steady:]]))
        x2 = np.abs(np.array([parse_julia_complex(val) for val in df["E2"].values[index_steady:]]))
        x3 = np.abs(np.array([parse_julia_complex(val) for val in df["E3"].values[index_steady:]]))
        x4 = np.abs(np.array([parse_julia_complex(val) for val in df["E4"].values[index_steady:]]))

        # Cross-correlation for each pair of lasers
        lags_12, corr_vals_12 = cross_correlation(x1, x2, lag_max_index)
        lags_13, corr_vals_13 = cross_correlation(x1, x3, lag_max_index)
        lags_14, corr_vals_14 = cross_correlation(x1, x4, lag_max_index)
        lags_23, corr_vals_23 = cross_correlation(x2, x3, lag_max_index)
        lags_24, corr_vals_24 = cross_correlation(x2, x4, lag_max_index)
        lags_34, corr_vals_34 = cross_correlation(x3, x4, lag_max_index)

        # Store results for each pair
        cross_correlation_results = {
            "12": (lags_12, corr_vals_12),
            "13": (lags_13, corr_vals_13),
            "14": (lags_14, corr_vals_14),
            "23": (lags_23, corr_vals_23),
            "24": (lags_24, corr_vals_24),
            "34": (lags_34, corr_vals_34),
        }


        for pair, (lags, corr_vals) in cross_correlation_results.items():
            c_max = np.max(np.abs(corr_vals))
            c_max_index = np.argmax(np.abs(corr_vals))
            lag_at_cmax = lags[c_max_index] * 1e9 * h * Δt  # in ns
            c_max_list.append((pair, c_max))
            lag_at_cmax_list.append((pair, lag_at_cmax))


    # Plot C_max vs Δf for all pairs together
    plt.figure()
    for pair in ["12", "13", "14", "23", "24", "34"]:
        pair_c_max = [c_max for p, c_max in c_max_list if p == pair]
        plt.plot(Δf_list, pair_c_max, marker='o', linewidth=1, label=f"Pair {pair}")
    plt.xlabel(r"$\Delta f$ (GHz)")
    plt.ylabel(r"$C_{\max}$")
    plt.ylim([0, 1])
    plt.grid(True)
    plt.legend()
    plt.savefig(os.path.join(current_dir, "c_max_vs_Δf_all_pairs_RK4.png"))

    # Print lag values at which C_max occurred
    print("Lag values at Cmax (in ns):", lag_at_cmax_list)

#Run analysis
analyze_cross_correlations()

## Calculation Autocorrelation function

"Without" normalization (np.correlate() but we divide by the first value, which is the maximum)

In [ ]:
def parse_julia_complex(x):
    return complex(x.replace("im", "j").replace(" ", ""))

def auto_corr_function():
    Δf_list = np.arange(-22, -19, 0.1)
    Δf_list = np.round(Δf_list, 1)  # Round to 1 decimal place
    for element in Δf_list:
        if element == -0.0:
            index = np.where(Δf_list == element)[0][0]
            Δf_list[index] = 0.0  # Change -0.0 to 0.0


    print(Δf_list)
    c_max_list = []
    lag_at_cmax_list = []
    pairs_list = []

    sigma_list_0 = []
    acf_area_list_0 = []

    sigma_list_1 = []
    acf_area_list_1 = []


    current_dir = os.getcwd()
    h = 0.2e-12
    Δt = 50
    tau = 1e-8
    lag_max_index = int(round(10*tau / (h * Δt)))  # max lag index (samples)

    for Δf in Δf_list:
        print(Δf)
        file_path = os.path.join(rf"/home/adam/vscode/python/Infolante/traces/results_4_lasers_new_params_larger_wait/4_lasers_simulation_Δf_{Δf}_RK4_no_noise_new_params_larger_wait.csv")

        df = pd.read_csv(file_path)
        t = df["time"].values
        index_steady = np.argmax(t > 19800e-9)

        x1 = np.abs(np.array([parse_julia_complex(val) for val in df["E1"].values[index_steady:]]))
        x2 = np.abs(np.array([parse_julia_complex(val) for val in df["E2"].values[index_steady:]]))

        # Cross-correlation for each pair of lasers
        lags_1, corr_vals_1 = cross_correlation(x1, x1, lag_max_index)
        corr_vals_1 = corr_vals_1[lag_max_index - 1: 2*lag_max_index -1]
        lags_1 = lags_1* 1e9 * h * Δt
        lags_1 = lags_1[lag_max_index - 1: 2*lag_max_index -1]

        lags_2, corr_vals_2 = cross_correlation(x2, x2, lag_max_index)
        corr_vals_2 = corr_vals_2[lag_max_index - 1: 2*lag_max_index -1]
        lags_2 = lags_2* 1e9 * h * Δt
        lags_2 = lags_2[lag_max_index - 1: 2*lag_max_index -1]

        area_under_acf_1 = np.trapezoid(np.abs(corr_vals_1), x =lags_1 )

        area_under_acf_2 = np.trapezoid(np.abs(corr_vals_2), x =lags_2 )

            #Plot C_max vs Δf for all pairs together
        plt.figure()
        plt.plot(lags_1, corr_vals_1, linewidth=1)
        plt.xlabel(r"lags(ns)")
        plt.ylabel(r"$ACF_{1}$")
        plt.title(rf"{Δf}, {area_under_acf_1}")

        plt.figure()
        plt.plot(lags_2, corr_vals_2, linewidth=1)
        plt.xlabel(r"lags(ns)")
        plt.ylabel(r"$ACF_{2}$")
        plt.title(rf"{Δf}, {area_under_acf_2}")



auto_corr_function()

## Classification of dynamics

In [ ]:
def parse_julia_complex(x):
    return complex(x.replace("im", "j").replace(" ", ""))

def analyze_cross_correlations():
    
    Δf_list = np.arange(-30, 30.1, 0.1)
    Δf_list = np.round(Δf_list, 1)  # Round to 1 decimal place
    for element in Δf_list:
        if element == -0.0:
            index = np.where(Δf_list == element)[0][0]
            Δf_list[index] = 0.0  # Change -0.0 to 0.0


    print(Δf_list)
    c_max_list = []
    lag_at_cmax_list = []
    pairs_list = []

    sigma_list_0 = []
    acf_area_list_0 = []

    sigma_list_1 = []
    acf_area_list_1 = []


    current_dir = os.getcwd()
    h = 0.2e-12
    Δt = 50
    tau = 1e-8
    lag_max_index = int(round(10*tau / (h * Δt)))  # max lag index (samples)

    for Δf in Δf_list:
        print(Δf)
        file_path = os.path.join(rf"/home/adam/vscode/python/Infolante/traces/results_4_lasers_new_params_larger_wait/4_lasers_simulation_Δf_{Δf}_RK4_no_noise_new_params_larger_wait.csv")

        df = pd.read_csv(file_path)
        t = df["time"].values
        index_steady = np.argmax(t > 19800e-9)
        index_steady_2 = np.argmax(t > 900e-9)

        x1 = np.abs(np.array([parse_julia_complex(val) for val in df["E1"].values[index_steady:]]))
        x2 = np.abs(np.array([parse_julia_complex(val) for val in df["E2"].values[index_steady:]]))

        x_1_lag = np.abs(np.array([parse_julia_complex(val) for val in df["E1"].values[index_steady-1000:-1000]]))

        #Then I should estimate the value for the delay and the embedding dimension

        # Cross-correlation for each pair of lasers
        lags_12, corr_vals_12 = cross_correlation(x1, x2, lag_max_index)
#        lags_13, corr_vals_13 = cross_correlation(x1, x3, lag_max_index)
#        lags_14, corr_vals_14 = cross_correlation(x1, x4, lag_max_index)
#        lags_23, corr_vals_23 = cross_correlation(x2, x3, lag_max_index)
#        lags_24, corr_vals_24 = cross_correlation(x2, x4, lag_max_index)
#        lags_34, corr_vals_34 = cross_correlation(x3, x4, lag_max_index)

        x_1_normalized = (x1 - np.mean(x1)) / np.std(x1)
        x_2_normalized = (x2 - np.mean(x2)) / np.std(x2)

        x_normalized = np.array((x_1_normalized,x_2_normalized))

        x = np.array((x1,x2))
        #print("Shape of x:", x.shape)
        sigma_normalized_list = np.zeros(2)
        for i in range(len(sigma_normalized_list)):
            sigma_normalized_list[i] = np.std(x[i,:])/np.mean(x[i,:])
        
        index = np.argmax(sigma_normalized_list)


        # Cross-correlation for each pair of lasers
        lags_1, corr_vals_1 = cross_correlation(x1, x1, lag_max_index)
        corr_vals_1 = corr_vals_1[lag_max_index - 1: 2*lag_max_index -1]
        lags_1 = lags_1* 1e9 * h * Δt
        lags_1 = lags_1[lag_max_index - 1: 2*lag_max_index -1]

        lags_2, corr_vals_2 = cross_correlation(x2, x2, lag_max_index)
        corr_vals_2 = corr_vals_2[lag_max_index - 1: 2*lag_max_index -1]
        lags_2 = lags_2* 1e9 * h * Δt
        lags_2 = lags_2[lag_max_index - 1: 2*lag_max_index -1]

        area_under_acf_1 = np.trapezoid(np.abs(corr_vals_1), x =lags_1 )

        area_under_acf_2 = np.trapezoid(np.abs(corr_vals_2), x =lags_2 )




        sigma_list_0.append(sigma_normalized_list[0])
        acf_area_list_0.append(area_under_acf_1)

        sigma_list_1.append(sigma_normalized_list[1])
        acf_area_list_1.append(area_under_acf_2)

        # Store results for each pair
        cross_correlation_results = {
            "12": (lags_12, corr_vals_12),
#            "13": (lags_13, corr_vals_13),
#            "14": (lags_14, corr_vals_14),
#            "23": (lags_23, corr_vals_23),
#            "24": (lags_24, corr_vals_24),
#            "34": (lags_34, corr_vals_34),
        }

#
        for pair, (lags, corr_vals) in cross_correlation_results.items():
            c_max = np.max(np.abs(corr_vals))
            c_max_index = np.argmax(np.abs(corr_vals))
            lag_at_cmax = lags[c_max_index] * 1e9 * h * Δt  # in ns
            c_max_list.append((pair, c_max))
            lag_at_cmax_list.append((pair, lag_at_cmax))

    # Plot C_max vs Δf for all pairs together
    plt.figure()
    i = 0
    for pair in ["12"]:
        pair_c_max = [c_max for p, c_max in c_max_list if p == pair]
        plt.plot(Δf_list, pair_c_max, linewidth=1, label=f"Pair {pair}", color = 'black')
        for i in range(0,len(Δf_list)):
            if sigma_list_0[i]<0.02 and acf_area_list_0[i]>58: #CW
                if sigma_list_1[i]<0.02 and acf_area_list_1[i]>58:#CW
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='s', linewidth=1, label=f"Pair {pair}", color = 'green')
                elif sigma_list_1[i]>0.02 and acf_area_list_1[i]<58: #Chaotic
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='o', linewidth=1, label=f"Pair {pair}", color = 'green')
                elif sigma_list_1[i]>0.02 and acf_area_list_1[i]>58: #Oscillatory
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='^', linewidth=1, label=f"Pair {pair}", color = 'green')
                else:
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='s', linewidth=1, label=f"Pair {pair}", color = 'green')


            elif sigma_list_0[i]>0.02 and acf_area_list_0[i]<58: #Chaotic
                if sigma_list_1[i]<0.02 and acf_area_list_1[i]>58:#CW
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='s', linewidth=1, label=f"Pair {pair}", color = 'blue')
                elif sigma_list_1[i]>0.02 and acf_area_list_1[i]<58: #Chaotic
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='o', linewidth=1, label=f"Pair {pair}", color = 'blue')
                elif sigma_list_1[i]>0.02 and acf_area_list_1[i]>58: #Oscillatory
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='^', linewidth=1, label=f"Pair {pair}", color = 'blue')
                else:
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='s', linewidth=1, label=f"Pair {pair}", color = 'blue')


            elif sigma_list_0[i]>0.02 and acf_area_list_0[i]>58: #oscillatory
                if sigma_list_1[i]<0.02 and acf_area_list_1[i]>58:#CW
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='s', linewidth=1, label=f"Pair {pair}", color = 'red')
                elif sigma_list_1[i]>0.02 and acf_area_list_1[i]<58: #Chaotic
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='o', linewidth=1, label=f"Pair {pair}", color = 'red')
                elif sigma_list_1[i]>0.02 and acf_area_list_1[i]>58: #Oscillatory
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='^', linewidth=1, label=f"Pair {pair}", color = 'red')
                else:
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='s', linewidth=1, label=f"Pair {pair}", color = 'red')

            else: #dunno
                if sigma_list_1[i]<0.02 and acf_area_list_1[i]>58:#CW
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='s', linewidth=1, label=f"Pair {pair}", color = 'green')
                elif sigma_list_1[i]>0.02 and acf_area_list_1[i]<58: #Chaotic
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='o', linewidth=1, label=f"Pair {pair}", color = 'green')
                elif sigma_list_1[i]>0.02 and acf_area_list_1[i]>58: #Oscillatory
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='^', linewidth=1, label=f"Pair {pair}", color = 'green')
                else:
                    plt.scatter(Δf_list[i], pair_c_max[i], marker='s', linewidth=1, label=f"Pair {pair}", color = 'green')
    
    plt.xlabel(r"$\Delta f$ (GHz)")
    plt.ylabel(r"$C_{\max}$")
    plt.ylim([0, 1])
    plt.savefig(os.path.join(current_dir, "c_max_vs_Δf_RK4_network_new_again_larger_wait_increased_changed_acf.png"))

    # Print lag values at which C_max occurred
    print("Lag values at Cmax (in ns):", lag_at_cmax_list)

    #Plot C_max vs Δf for all pairs together
    plt.figure()
    plt.plot(Δf_list, sigma_list_0, marker='o', linewidth=1)
    plt.plot(Δf_list, sigma_list_1, marker='^', linewidth=1)
    plt.xlabel(r"$\Delta f$ (GHz)")
    plt.ylabel(r"$\sigma/\mu$")
    plt.savefig(os.path.join(current_dir, "simga_mu_vs_Δf_RK4_network_new_again_larger_wait_increased_changed_acf.png"))

    plt.figure()
    plt.plot(Δf_list, acf_area_list_0, marker='o', linewidth=1)
    plt.plot(Δf_list, acf_area_list_1, marker='^', linewidth=1)
    plt.xlabel(r"$\Delta f$ (GHz)")
    plt.ylabel(r"$|ACF_{area}|$")
    plt.savefig(os.path.join(current_dir, "acf_area_vs_Δf_RK4_network_new_again_larger_wait_increased_changed_acf.png"))

#Run analysis
analyze_cross_correlations()